In [1]:
!pip install mistralai -U
!pip install python-dotenv
!pip3 install spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [1]:
import json
import os
from dotenv import load_dotenv
load_dotenv()
from IPython.display import display, Markdown

In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
import requests

def get_current_weather(city, api_key=os.getenv("OPENWEATHER_API_KEY"), unit="metric"):
    """Get the current weather for a given city using OpenWeather API."""
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": unit
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        #print(data)
        weather = {
            "location": data["name"],
            "temperature": int(data["main"]["temp"]),
            "rain": data["weather"][0]["main"],
            "unit": "Celsius" if unit == "metric" else "Fahrenheit"
        }
        return json.dumps(weather)
    else:
        return {"location": city, "temperature": "unknown", "unit": unit, "rain": "unknown"}

api_key = os.getenv("OPENWEATHER_API_KEY")
weather_info = get_current_weather("Nagercoil")
print(weather_info)

{"location": "Nagercoil", "temperature": 29, "rain": "Clouds", "unit": "Celsius"}


In [31]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from IPython.display import Image, display, Audio

def search_song(song_name):
    # Set up your Spotify credentials
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

    # Authenticate with Spotify
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Search for the song
    results = sp.search(q=song_name, limit=1, type='track')
    tracks = results['tracks']['items']

    # Display the first result
    if tracks:
        track = tracks[0]
        return json.dumps({
            "song": track['name'],
            "artist": ', '.join(artist['name'] for artist in track['artists']),
            "album": track['album']['name'],
            "album_cover_url": track['album']['images'][0]['url'],
            "preview_url": track['preview_url']
        })

    else:
        return "No song found"

song_name = "arabi kuthu"
song_details = search_song(song_name)
print(song_details)

{"song": "Arabic Kuthu - Halamithi Habibo (From \"Beast\")", "artist": "Anirudh Ravichander, Jonita Gandhi", "album": "Arabic Kuthu - Halamithi Habibo (From \"Beast\")", "album_cover_url": "https://i.scdn.co/image/ab67616d0000b273b6b3b7f26f0bc0e0197163a0", "preview_url": "https://p.scdn.co/mp3-preview/fa5ac33ff747d7b71a99d6ad9ca4a3aaa1cc5f1b?cid=5385f207ed2d4b01ad1becda6e213329"}


In [24]:

available_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location, use farhenheit",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather for",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to use for the temperature, metric is default",
                        "enum": ["metric", "imperial"],
                    }
                },
                "required": ["city", "unit"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_song",
            "description": "Search for a song on Spotify and display its details including the artist, album, album cover, and a preview link if available",
            "parameters": {
                "type": "object",
                "properties": {
                    "song_name": {
                        "type": "string",
                        "description": "The name of the song to search for",
                    }
                },
                "required": ["song_name"],
            }
        }
    }
]

names_to_functions = {
    'get_current_weather': get_current_weather,
    'search_song': search_song
}

In [25]:
client = MistralClient(api_key=api_key)

In [44]:

def get_weather(city):
    messages = [
        ChatMessage(role="user", content=f"What's the current weather in {city}?")
    ]

    model = "mistral-large-latest"
    api_key = os.getenv("MISTRAL_API_KEY")

    client = MistralClient(api_key=api_key)
    response = client.chat(model=model, messages=messages, tools=available_tools, tool_choice="auto")

    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    function_params = json.loads(tool_call.function.arguments)

    function_result = names_to_functions[function_name](**function_params)
    return json.loads(function_result)

def suggest_songs(weather_info):
    messages = [
        ChatMessage(role="system", content="A Song Suggestions Assistant based on local weather and language"),
        ChatMessage(role="user", content=f"""The current weather is {weather_info['temperature']} degrees {weather_info['unit']} with {weather_info['rain']} in {weather_info['location']}.
        Suggest 5 songs based on this weather and the local language.
        Return a json object with keys as song, artist, album, album_cover_url, preview_url and reason.
        Use the key 'song_suggestions' and provide the value as a list of json objects with keys song, artist, album, album_cover_url, preview_url and reason.""")
    ]

    model = "mistral-large-latest"
    api_key = os.getenv("MISTRAL_API_KEY")

    client = MistralClient(api_key=api_key)
    response = client.chat(model=model, messages=messages, tools=available_tools, tool_choice="auto")

    while response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        function_params = json.loads(tool_call.function.arguments)

        function_result = names_to_functions[function_name](**function_params)

        messages.append(ChatMessage(role="tool", name=function_name, content=function_result))
        response = client.chat(model=model, messages=messages, tools=available_tools, tool_choice="auto")

    return response

city = "Sydney"
weather_info = get_weather(city)
result = suggest_songs(weather_info)
song_suggestions =result.choices[0].message.content

# for song in song_suggestions_json["song_suggestions"]:
#     print(f"### {song['song']}")
#     print(f"**Artist:** {song['artist']}")
#     print(f"**Album:** {song['album']}")
#     print(f"[Preview the song]({song['preview_url']})")
#     print(f"**Reason:** {song['reason']}")
#     print("----")

In [45]:
model = "mistral-large-latest"
api_key = os.getenv("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)



In [46]:
final_res = f"""Convert it into a json object with keys as song, artist, album, album_cover_url, preview_url and reason.
Use the key 'song_suggestions' and provide the value as a list of json objects with keys song, artist, album, album_cover_url, preview_url and reason JSON format.

Message:
{result.choices[0].message.content}

"""

# mistral call

result = client.chat(model="mistral-large-latest", response_format={"type": "json_object"},messages=[ChatMessage(role="user", content=final_res)])

print(result.choices[0].message.content)



{"song_suggestions": [{"song": "Sunny", "artist": "Bobby Hebb", "album": "Sunny (Remastered 2014)", "album_cover_url": "https://i.scdn.co/image/ab67616d0000b273e97c6646fc7407175c999e4d", "preview_url": "https://p.scdn.co/mp3-preview/7acdb7876e48d68266b522b83b742e1e18ebc237?cid=774b29d4f13844c495f206cafdad9c86", "reason": "This classic upbeat song matches the warm and sunny weather in Sydney."}, {"song": "Three Little Birds", "artist": "Bob Marley & The Wailers", "album": "Exodus (Deluxe Edition)", "album_cover_url": "https://i.scdn.co/image/ab67616d0000b2736e4b9f5b47261e9f9570287b", "preview_url": "https://p.scdn.co/mp3-preview/3e2a4444751263f12a039e3416c6e27b6d262745?cid=774b29d4f13844c495f206cafdad9c86", "reason": "This relaxing song with a positive message suits the pleasant weather in Sydney."}, {"song": "Walking on Sunshine", "artist": "Katrina & The Waves", "album": "Walking on Sunshine", "album_cover_url": "https://i.scdn.co/image/ab67616d0000b27327e97b2969b70f1951d56636", "prev

In [29]:
# print json
song_suggestions = json.loads(result.choices[0].message.content)
for song in song_suggestions["song_suggestions"]:
    print(song)



{'song': 'Kadhal Rojave', 'artist': 'Sid Sriram', 'album': 'Chennai 2 Singapore', 'album_cover_url': 'https://i.scdn.co/image/ab67616d0000b2730096db8d86803161529b63c6', 'preview_url': 'https://p.scdn.co/mp3-preview/7529a65f6b56e98b5f7700b13a446504b01b39f1?cid=774b29d4f13844c495f206cafdad9c86', 'reason': 'This is a popular Tamil song with a lively tune that suits the hot, cloudy weather.'}
{'song': 'Ennodu Nee Irundhal', 'artist': 'Anirudh Ravichander, Shakthisree Gopalan', 'album': 'I', 'album_cover_url': 'https://i.scdn.co/image/ab67616d0000b27337e8e0e36598ded68e3f0905', 'preview_url': 'https://p.scdn.co/mp3-preview/a50e6560e2c660f08a94ab24795b3658424e9362?cid=774b29d4f13844c495f206cafdad9c86', 'reason': 'This song has a romantic, soothing melody that matches the cloudy sky.'}


In [35]:
for song in song_suggestions["song_suggestions"]:
    print(f"### {song['song']}")
    print(f"**Artist:** {song['artist']}")
    print(f"**Album:** {song['album']}")
    print(f"[Preview the song]({song['preview_url']})")
    #print(f"[Preview the song]({song['additional_info']['preview_url']})")
    print(f"**Reason:** {song['reason']}")
    print("----")

### Kadhal Rojave
**Artist:** Sid Sriram
**Album:** Chennai 2 Singapore
[Preview the song](https://p.scdn.co/mp3-preview/7529a65f6b56e98b5f7700b13a446504b01b39f1?cid=774b29d4f13844c495f206cafdad9c86)
**Reason:** This is a popular Tamil song with a lively tune that suits the hot, cloudy weather.
----
### Ennodu Nee Irundhal
**Artist:** Anirudh Ravichander, Shakthisree Gopalan
**Album:** I
[Preview the song](https://p.scdn.co/mp3-preview/a50e6560e2c660f08a94ab24795b3658424e9362?cid=774b29d4f13844c495f206cafdad9c86)
**Reason:** This song has a romantic, soothing melody that matches the cloudy sky.
----


In [51]:
def generate_weather_music(city):
    messages = [
        {
            "role": "system",
            "content": "A Song Suggestions Assistant based on local weather in metric and local language"
        },
        {
            "role": "user",
            "content": f"""I am in {city}, suggest 5 songs based on their current weather and their local language
            and display their album details such as album cover, artist, and preview link.
            return a json object with keys as song, artist, album, album_cover_url, and preview_url and reason

            key as song_suggestions and value as a list of json objects with keys as song, artist, album, album_cover_url, preview_url, and reason
            """
        }
    ]
    tools = available_tools
    response = client.chat(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="any",

    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        function_name = tool_calls[0].function.name
        function_args = json.loads(tool_calls[0].function.arguments)
        function_response = names_to_functions[function_name](**function_args)

        messages.append(response_message)
        messages.append(
            {
                "tool_call_id": tool_calls[0].id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

        second_response = client.chat(
            model=model,
            messages=messages,
            tools=available_tools,
            tool_choice="any",
        )

        response_message = second_response.choices[0].message
        tool_calls = response_message.tool_calls

        if tool_calls:
            messages.append(response_message)
            for tool in tool_calls:
                function_name = tool.function.name
                function_args = json.loads(tool.function.arguments)
                function_response = names_to_functions[function_name](**function_args)

                messages.append(
                    {
                        "tool_call_id": tool.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            third_response = client.chat(
                model= model,
                messages=messages,
                response_format={"type":"json_object"}
            )

            return third_response

In [58]:
result = generate_weather_music("San Francisco")

In [59]:
song_suggestions_json = result.choices[0].message.content

In [60]:
type(song_suggestions_json)

str

In [61]:
song_suggestions_json = json.loads(song_suggestions_json)

In [62]:
song_suggestions_json

{'song_suggestions': [{'song': 'Rainy Day Song',
   'artist': 'Andrew Combs',
   'album': 'All These Dreams',
   'album_cover_url': 'https://i.scdn.co/image/ab67616d0000b2739aed7bc3a3b1bb2c5ac5ad24',
   'preview_url': 'https://p.scdn.co/mp3-preview/ad03880dc1f0e2a89d81dc7e400f50c607d86492?cid=5385f207ed2d4b01ad1becda6e213329',
   'reason': "Since it's a bit chilly and cloudy in San Francisco, this song fits the mood."},
  {'song': 'The Cloudy Day',
   'artist': 'Jeff Pitchell',
   'album': "Rockin' Songs for Kids",
   'album_cover_url': 'https://i.scdn.co/image/ab67616d0000b27331376cc8d2c7bdf6d6c53bbb',
   'preview_url': 'https://p.scdn.co/mp3-preview/ddeb96bebe814a76ecb6762777e9f8404086546b?cid=5385f207ed2d4b01ad1becda6e213329',
   'reason': "It's a cloudy day in San Francisco, this song matches the weather."},
  {'song': 'San Francisco Song',
   'artist': 'Camera Obscura',
   'album': 'Keep It Clean',
   'album_cover_url': 'https://i.scdn.co/image/ab67616d0000b273c56fd4d88e853aa917e1

In [63]:
for song in song_suggestions_json["song_suggestions"]:
    display(Markdown(f"### {song['song']}"))
    display(Markdown(f"**Artist:** {song['artist']}"))
    display(Markdown(f"**Album:** {song['album']}"))
    display(Image(url=song['album_cover_url'], width=200))
    display(Markdown(f"[Preview the song]({song['preview_url']})"))
    display(Markdown(f"**Reason:** {song['reason']}"))
    display(Markdown("----"))

### Rainy Day Song

**Artist:** Andrew Combs

**Album:** All These Dreams

[Preview the song](https://p.scdn.co/mp3-preview/ad03880dc1f0e2a89d81dc7e400f50c607d86492?cid=5385f207ed2d4b01ad1becda6e213329)

**Reason:** Since it's a bit chilly and cloudy in San Francisco, this song fits the mood.

...

**Artist:** Cold War Kids

**Album:** Locker Room Talk (30 Days, 30 Songs)

[Preview the song](https://p.scdn.co/mp3-preview/563a63a2aca2ae8b95bed48c0461ab93b3575932?cid=5385f207ed2d4b01ad1becda6e213329)

**Reason:** A song that has 'cold' in the title, perfect for the cool weather in San Francisco.

----